![spotify_logo](images/spotify_logo2.1.png)

# Spotify API Scrape

## Project Goal
1. Pull most recently played tracks
2. Pull top listen to tracks

## Helpful Links:
- [Spotify Web API - Authorization Guide](https://developer.spotify.com/documentation/general/guides/authorization-guide/)
- [Spotify API References](https://developer.spotify.com/documentation/web-api/reference/)

In [1]:
import config
import os
import requests
import json
from json import JSONEncoder
import pandas as pd
from datetime import datetime,timezone, timedelta
from tqdm import tqdm
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import base64
from urllib.parse import urlencode

import pytz
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import xlsxwriter
from tzlocal import get_localzone
import dateutil
import pandas_gbq
from google.oauth2 import service_account
import pipdeptree

In [2]:
credentials = service_account.Credentials.from_service_account_file(
    r'/Users/Nicholas/Desktop/GCP/GCP_Keys/spotify-project-287802-e5e6c43e8ecb.json'
)

In [3]:
# Create the file name we will be exporting later
today = datetime.today().strftime('%Y%m%d')
file_name = f"Spotify_Export_{today}.xlsx"
file_name

'Spotify_Export_20201030.xlsx'

## Step 1: Get Access Token

In [4]:
client_id = config.client_id
client_secret = config.client_secret
username = config.username

In [5]:
username = config.username
client_id = config.client_id
client_secret = config.client_secret
redirect_uri = 'http://localhost:7777/callback'
scope = 'user-read-recently-played'

auth_token = util.prompt_for_user_token(username=username, 
                                   scope=scope, 
                                   client_id=client_id,   
                                   client_secret=client_secret,     
                                   redirect_uri=redirect_uri)

## Step 2: Pull Recently Played

In [ ]:
base_url = 'https://api.spotify.com/v1/me/player/recently-played?'
#track_id = '6y0igZArWVi6Iz0rj35c1Y'

#2. Authentication
#3. Parameters -- would be stored with authentication
headers = {
    "Authorization": f"Bearer {auth_token}"
}

#4. Create an empty list
personal_data = [] #would be good explore how to capture data at different points in time
r = requests.get(base_url+"&limit=50", headers=headers)
personal_data.append(json.loads(r.text))

In [ ]:
personal_data[0]['items'][3]['track']['artists'][0]['id']

In [ ]:
track_ids = []
album_ids = []
artist_ids = []
album_names = []
artist_names = []
track_names = []
popularity_ls = []
played_ats = []

for i in range(len(personal_data[0]['items'])):
    track_ids.append(personal_data[0]['items'][i]['track']['id']) # Track ID
    album_ids.append(personal_data[0]['items'][i]['track']['album']['id']) # Albumn ID
    artist_ids.append(personal_data[0]['items'][i]['track']['artists'][0]['id']) # Artist ID
    album_names.append(personal_data[0]['items'][i]['track']['album']['name']) # Album Name
    artist_names.append(personal_data[0]['items'][i]['track']['artists'][0]['name']) # Artist Name
    track_names.append(personal_data[0]['items'][i]['track']['name']) # Track Name
    popularity_ls.append(personal_data[0]['items'][i]['track']['popularity']) # Track Popularity
    played_ats.append(personal_data[0]['items'][i]['played_at'])

In [ ]:
list_dic={'track_id':track_ids,
          'album_id':album_ids,
          'artist_id':artist_ids,
          'track_name':track_names,
          'artist_name':artist_names,
          'album_name':album_names,
          'track_popularity':popularity_ls,
          'time_palyed': played_ats
    }

In [ ]:
df1=pd.DataFrame(list_dic)
df1

In [ ]:
# Create empty column to append data to
df1['local_time'] = ''

for i in range(len(personal_data[0]['items'])):
    # Convert UTC to local time zone
    utc_time = dateutil.parser.parse(df1['time_palyed'].iloc[i]).astimezone(get_localzone())
    # Format date/time
    local_time= utc_time.strftime('%Y-%m-%d %H:%M:%S')
    local_time
    
    df1['local_time'].iloc[i] = local_time

In [ ]:
df1.head()

## Check GCP BQ and Drop Any Duplicates (time_played)

In [ ]:
# Query the BQ table for case_ids
project_id = "spotify-project-287802"

sql = """
SELECT time_palyed
FROM `spotify-project-287802.spotify_api.recently_played_tracks` 
LIMIT 100
"""
# Load table into df
bq_df = pandas_gbq.read_gbq(sql, project_id=project_id, credentials=credentials)

# Create list of dq_df case_ids
case_to_drop = bq_df['time_palyed'].tolist()
# Compare case_ids from df1 if already in case_to_drop list
df1_clean = df1[~df1['time_palyed'].str.contains('|'.join(case_to_drop))]
print('After duplicate check, there are now {} new Case IDs from the API.'.format(len(df1_clean['time_palyed'])))

In [ ]:
df1_clean

## Load into GCP Big Query

In [ ]:
print('Loading data into GCP BigQuery')
# Load into GCP BigQuery
# Connect to Google Cloud API and upload dataframe
destinatoin_table = 'spotify_api.recently_played_tracks'
project_id = 'spotify-project-287802'


pandas_gbq.to_gbq(df1_clean, destinatoin_table, project_id, if_exists='append', 
                  credentials=credentials)
print('Script is complete; check table for details.')

In [ ]:
writer = pd.ExcelWriter(file_name)
df1.to_excel(writer,'recently_played')

## Step 3: Pull User's Top Artists & Tracks

In [ ]:
username = config.username
client_id = config.client_id
client_secret = config.client_secret
redirect_uri = 'http://localhost:7777/callback'
scope = 'user-top-read'

auth_token = util.prompt_for_user_token(username=username, 
                                   scope=scope, 
                                   client_id=client_id,   
                                   client_secret=client_secret,     
                                   redirect_uri=redirect_uri)

In [ ]:
base_url = 'https://api.spotify.com/v1/me/top/tracks?'
#track_id = '6y0igZArWVi6Iz0rj35c1Y'

#2. Authentication
#3. Parameters -- would be stored with authentication
headers = {
    "Authorization": f"Bearer {auth_token}"
}

#4. Create an empty list
top_track_data = [] #would be good explore how to capture data at different points in time
r = requests.get(base_url+"time_range=medium_term"+"&&limit=50", headers=headers)
top_track_data.append(json.loads(r.text))

In [ ]:
track_idss = []
album_idss = []
artist_idss = []
album_namess = []
album_relase_datess = []
artist_namess = []
popularity_lss = []
track_namess = []

for i in range(len(top_track_data[0]['items'])):
    track_idss.append(top_track_data[0]['items'][i]['id']) # Track ID
    album_idss.append(top_track_data[0]['items'][i]['album']['id']) # Album ID
    artist_idss.append(top_track_data[0]['items'][i]['album']['artists'][0]['id'])
    album_namess.append(top_track_data[0]['items'][i]['album']['name']) # Album Name
    album_relase_datess.append(top_track_data[0]['items'][i]['album']['release_date'])
    artist_namess.append(top_track_data[0]['items'][i]['album']['artists'][0]['name']) # Artist Name
    popularity_lss.append(top_track_data[0]['items'][i]['popularity'])
    track_namess.append(top_track_data[0]['items'][i]['name']) # Track Name

In [ ]:
list_dic2={'track_id':track_idss,
           'album_id':album_idss,
           'artist_id':artist_idss,
           'track_name':track_namess,
           'album_name':album_namess,
           'artist_name':artist_namess,
           'track_popularity':popularity_lss,
           'album_relase_date':album_relase_datess,
    }

In [ ]:
df2=pd.DataFrame(list_dic2)
df2

## Step 4: Pull Track Info
- API Doc: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-analysis/

## TO DO: Create table in GCP for track info

In [ ]:
# Query GCP BQ for top 5 played tracks

project_id = "spotify-project-287802"

sql = """
SELECT track_id
    , album_id
    , artist_id
    , track_name
    , count(track_id) AS track_count
    , artist_name
    , album_name
FROM `spotify-project-287802.spotify_api.recently_played_tracks`
WHERE CAST(CAST(local_time AS TIMESTAMP) AS DATE) >= DATE_SUB(CURRENT_DATE(), INTERVAL 30 DAY)
GROUP BY track_id
    , album_id
    , artist_id
    , track_name
    , artist_name
    , album_name
order by track_count DESC 
"""
# Load table into df
bq_df2 = pandas_gbq.read_gbq(sql, project_id=project_id, credentials=credentials)

bq_df2.head()

### Check for duplicates before adding songs to spotify playlist

In [ ]:
# Query the BQ table for case_ids
project_id = "spotify-project-287802"

sql = """
SELECT track_id
FROM `spotify-project-287802.spotify_api.spotify_python_playlist` 
"""
# Load table into df
track_check = pandas_gbq.read_gbq(sql, project_id=project_id, credentials=credentials)

# Create list of dq_df case_ids
case_to_drop = track_check['track_id'].tolist()
# Compare case_ids from df1 if already in case_to_drop list
bq_df2_clean = bq_df2[~bq_df2['track_id'].str.contains('|'.join(case_to_drop))]
print('After duplicate check, there are now {} new Case IDs from the API.'.format(len(bq_df2_clean['track_id'])))

In [ ]:
top5_mtracks = bq_df2_clean.iloc[:5]
top5_mtracks

In [ ]:
# load data into GCP for later use
print('Loading data into GCP BigQuery')
# Load into GCP BigQuery
# Connect to Google Cloud API and upload dataframe
destinatoin_table = 'spotify_api.spotify_python_playlist'
project_id = 'spotify-project-287802'


pandas_gbq.to_gbq(top5_mtracks, destinatoin_table, project_id, if_exists='append', 
                  credentials=credentials)
print('Script is complete; check table for details.')

### Create a list of track ids of the top 5 songs of the month

In [ ]:
track_id_ls=top5_mtracks['track_id'].tolist()
track_id_ls

### Pass list to get audio features

In [ ]:
track_data = []
for id in tqdm(track_id_ls):
    base_url = f'https://api.spotify.com/v1/audio-features/{id}?'

    #2. Authentication
    #3. Parameters -- would be stored with authentication
    headers = {
        "Authorization": f"Bearer {auth_token}"
    }

    r = requests.get(base_url, headers=headers)
    track_data.append(json.loads(r.text))

In [ ]:
track_df = pd.json_normalize(track_data)
track_df.head()

In [ ]:
# Used to post songs to playlist
spotify_urls_ls = track_df['uri'].tolist()
spotify_urls_ls

## Step 5: Pull Artist Info
- [Artist Endpoint Documentation](https://developer.spotify.com/documentation/web-api/reference/artists/get-artist/)

## TO DO: Create table in GCP for artist info

In [ ]:
# Creating a uniqe list artist ids
artist_id_ls = df2['artist_id'].tolist()
artist_id_ls = list(dict.fromkeys(artist_id_ls))

artist_id_ls

In [ ]:
artist_data = []

for a_id in tqdm(artist_id_ls):
    base_url = f'https://api.spotify.com/v1/artists/{a_id}?'
    # example artist_id = '06HL4z0CvFAxyc27GXpf02'

    #2. Authentication
    #3. Parameters -- would be stored with authentication
    headers = {
        "Authorization": f"Bearer {auth_token}"
    }

    #4. Create an empty list
    r = requests.get(base_url, headers=headers)
    artist_data.append(json.loads(r.text))

In [ ]:
artist_data

In [ ]:
len(artist_data[7]['genres'])

In [ ]:
artist_id_ls = []
artist_followers_ls = []
artist_genres_ls = []
artist_name_ls = []
artist_popularity_ls = []

for i in tqdm(range(len(artist_data))):
    artist_id_ls.append(artist_data[i]['id'])
    artist_followers_ls.append(artist_data[i]['followers']['total'])
    artist_name_ls.append(artist_data[i]['name'])
    artist_popularity_ls.append(artist_data[i]['popularity'])
    
    if len(artist_data[i]['genres']) > 0:
        artist_genres_ls.append(artist_data[i]['genres'][0])
    else:
        artist_genres_ls.append('not listed')

In [ ]:
list_dic3={'artist_id':artist_id_ls,
           'artist_followers':artist_followers_ls,
           'artist_genres':artist_genres_ls,
           'artist_name':artist_name_ls,
           'artist_popularity':artist_popularity_ls
    }

In [ ]:
artist_df = pd.DataFrame(list_dic3)
artist_df

## Clean Up Track Data

## Update 'mode' to tell if track is major or minor

In [ ]:
mod_dict = {0 : 'Minor',
            1: 'Major'}

In [ ]:
track_df['mode'].replace(mod_dict, inplace=True)

## Update "key" to tell the actually key

In [ ]:
music_dic = {
    0: 'C',
    1: 'C#/Db',
    2: 'D',
    3: 'D#/Eb',
    4: 'E',
    5: 'F',
    6: 'F#/Gb',
    7: 'G',
    8: 'G#/Ab',
    9: 'A',
    10: 'A#/Bb',
    11: 'B' 
}

In [ ]:
track_df['key'].replace(music_dic, inplace=True)

## Add column duration in minutes/seconds

In [ ]:
track_df['track_duration'] = ''

for i in range(len(track_df['id'])):
    millis=track_df['duration_ms'].iloc[i]
    track_df['track_duration'].iloc[i] = pd.to_datetime(millis, unit='ms').strftime('%H:%M:%S')

## Rename track id column

In [ ]:
track_df=track_df.rename(columns = {'id':'track_id'})
track_df.head()                     

# Create Master Dataframe

In [ ]:
master_df = pd.merge(df2, track_df,
                       how='left', on=['track_id'])

In [ ]:
master_df.head(10)

In [ ]:
master_df.to_excel(writer,'top_tracks')

# Create Excel File

In [ ]:
current_directory = os.path.abspath(os.getcwd())

In [ ]:
export_file_path = os.path.join(current_directory, 'spotify_export_files', file_name)

In [ ]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(export_file_path, engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df1.to_excel(writer,'recently_played', index=False)
master_df.to_excel(writer,'top_tracks',index=False)

# Get the xlsxwriter workbook and worksheet objects.
workbook  = writer.book
worksheet1 = writer.sheets['recently_played']
worksheet2 = writer.sheets['top_tracks']


for i, col in enumerate(master_df.columns):
    # find length of column i
    column_len = master_df[col].astype(str).str.len().max()
    # Setting the length if the column header is larger
    # than the max column value length
    column_len = max(column_len, len(col)) + 2
    
    # set the column length
    worksheet1.set_column(i, i, column_len)
    
for i, col in enumerate(df1.columns):
    # find length of column i
    column_len = df1[col].astype(str).str.len().max()
    # Setting the length if the column header is larger
    # than the max column value length
    column_len = max(column_len, len(col)) + 2
    
    # set the column length
    worksheet2.set_column(i, i, column_len)

writer.save()

# Check to see if playlist exist

## Pull user playlist info

In [28]:
username = config.username
client_id = config.client_id
client_secret = config.client_secret
redirect_uri = 'http://localhost:7777/callback'
scope = 'playlist-read-collaborative, playlist-read-private, playlist-modify-public, playlist-modify-private'

auth_token = util.prompt_for_user_token(username=username, 
                                   scope=scope, 
                                   client_id=client_id,   
                                   client_secret=client_secret,     
                                   redirect_uri=redirect_uri)

In [29]:
base_url = f'https://api.spotify.com/v1/users/{username}/playlists?'

#2. Authentication
#3. Parameters -- would be stored with authentication
headers = {
    "Authorization": f"Bearer {auth_token}"
}

#4. Create an empty list
personal_playlist_data = [] #would be good explore how to capture data at different points in time
r = requests.get(base_url+"&limit=50", headers=headers)
personal_playlist_data.append(json.loads(r.text))

In [30]:
playlist_ids = []
playlist_names = []
playlist_descriptions = []
playlist_owners = []
playlist_publics = []

for i in range(len(personal_playlist_data[0]['items'])):
    playlist_ids.append(personal_playlist_data[0]['items'][i]['id'])
    playlist_names.append(personal_playlist_data[0]['items'][i]['name'])
    playlist_descriptions.append(personal_playlist_data[0]['items'][i]['description'])
    playlist_owners.append(personal_playlist_data[0]['items'][i]['owner']['display_name'])
    playlist_publics.append(personal_playlist_data[0]['items'][i]['public'])

In [31]:
list_dic4={'playlist_id':playlist_ids,
           'playlist_name':playlist_names,
           'playlist_description':playlist_descriptions,
           'playlist_owner':playlist_owners,
           'playlist_public':playlist_publics
    }

In [32]:
playlist_df = pd.DataFrame(list_dic4)
playlist_df.head()

,playlist_id,playlist_name,playlist_description,playlist_owner,playlist_public
0,2tmzVHK0OfcbWWViZve0B6,Fall 2020,,Nicholas Drake,True
1,79tXL4BD7MnqvpMZtKDho9,Test Python Playlist 2020,This is a test playlist generated by Python,Nicholas Drake,True
2,7Esbky8lu1dZKcAMcovavP,Michelle’s Queued Up Discovery,,Nicholas Drake,False
3,5b4Ko7aAOpk8JT9qQnngOy,Pop Punk,,Amber Layn Miller,False
4,0RSKurPBF9Jd2W6zjAw6ZI,Chicago Unite At Night Complete Collection,Chicago nightly sing-along collection during C...,Chicago Unite At Night,False


## Check to verify playlist exist
If it doesn't, create it

In [45]:
cur_year = datetime.today().strftime('%Y')
cur_year

'2020'

In [35]:
if f'Test Python Playlist {cur_year}' in set(playlist_df['playlist_name']):
    print("Playlist exist!")
    
    base_url = f'https://api.spotify.com/v1/users/{username}/playlists?'

    #2. Authentication
    #3. Parameters -- would be stored with authentication
    headers = {
        "Authorization": f"Bearer {auth_token}"
    }

    #4. Create an empty list
    personal_playlist_data = [] #would be good explore how to capture data at different points in time
    r = requests.get(base_url+"&limit=50", headers=headers)
    personal_playlist_data.append(json.loads(r.text))
    
    playlist_ids = []
    playlist_names = []
    playlist_descriptions = []
    playlist_owners = []
    playlist_publics = []

    for i in range(len(personal_playlist_data[0]['items'])):
        playlist_ids.append(personal_playlist_data[0]['items'][i]['id'])
        playlist_names.append(personal_playlist_data[0]['items'][i]['name'])
        playlist_descriptions.append(personal_playlist_data[0]['items'][i]['description'])
        playlist_owners.append(personal_playlist_data[0]['items'][i]['owner']['display_name'])
        playlist_publics.append(personal_playlist_data[0]['items'][i]['public'])
        
    list_dic4={'playlist_id':playlist_ids,
           'playlist_name':playlist_names,
           'playlist_description':playlist_descriptions,
           'playlist_owner':playlist_owners,
           'playlist_public':playlist_publics
    }
    
    playlist_df = pd.DataFrame(list_dic4)
    playlist_df.head()
    
    
    
else:
    print("Creating Playlist...")
    base_url = f'https://api.spotify.com/v1/users/{username}/playlists'

    #2. Authentication
    #3. Parameters -- would be stored with authentication
    headers = {
        "Content-Type":"application/json",
        "Authorization": f"Bearer {auth_token}"
    }

    request_body = json.dumps({
              "name": f"Test Python Playlist {cur_year}",
              "description": f"This is a test playlist generated by Python for {cur_year}.",
              "public": False # private
            })
    #4. Create an empty list
    r = requests.post(url = base_url, data = request_body, headers=headers)
    print(r.text)

Playlist exist!


# How to Create Playlist

In [ ]:
username = config.username
client_id = config.client_id
client_secret = config.client_secret
redirect_uri = 'http://localhost:7777/callback'
scope = 'playlist-modify-private'

auth_token = util.prompt_for_user_token(username=username, 
                                   scope=scope, 
                                   client_id=client_id,   
                                   client_secret=client_secret,     
                                   redirect_uri=redirect_uri)

In [ ]:
base_url = f'https://api.spotify.com/v1/users/{username}/playlists'

#2. Authentication
#3. Parameters -- would be stored with authentication
headers = {
    "Content-Type":"application/json",
    "Authorization": f"Bearer {auth_token}"
}

request_body = json.dumps({
          "name": "Test Python Playlist",
          "description": "This is a test playlist generated by Python.",
          "public": False # private
        })
#4. Create an empty list
r = requests.post(url = base_url, data = request_body, headers=headers)
print(r.text)

In [ ]:
print(r.text)

# Pull Playlist ID

In [36]:
username = config.username
client_id = config.client_id
client_secret = config.client_secret
redirect_uri = 'http://localhost:7777/callback'
scope = 'playlist-read-private'

auth_token = util.prompt_for_user_token(username=username, 
                                   scope=scope, 
                                   client_id=client_id,   
                                   client_secret=client_secret,     
                                   redirect_uri=redirect_uri)

In [37]:
base_url = f'https://api.spotify.com/v1/users/{username}/playlists?'

#2. Authentication
#3. Parameters -- would be stored with authentication
headers = {
    "Authorization": f"Bearer {auth_token}"
}

#4. Create an empty list
personal_playlist_data = [] #would be good explore how to capture data at different points in time
r = requests.get(base_url+"&limit=50", headers=headers)
personal_playlist_data.append(json.loads(r.text))

In [38]:
playlist_ids = []
playlist_names = []
playlist_descriptions = []
playlist_owners = []
playlist_publics = []

for i in range(len(personal_playlist_data[0]['items'])):
    playlist_ids.append(personal_playlist_data[0]['items'][i]['id'])
    playlist_names.append(personal_playlist_data[0]['items'][i]['name'])
    playlist_descriptions.append(personal_playlist_data[0]['items'][i]['description'])
    playlist_owners.append(personal_playlist_data[0]['items'][i]['owner']['display_name'])
    playlist_publics.append(personal_playlist_data[0]['items'][i]['public'])

In [39]:
list_dic4={'playlist_id':playlist_ids,
           'playlist_name':playlist_names,
           'playlist_description':playlist_descriptions,
           'playlist_owner':playlist_owners,
           'playlist_public':playlist_publics
    }

In [40]:
playlist_df = pd.DataFrame(list_dic4)
playlist_df.head()

,playlist_id,playlist_name,playlist_description,playlist_owner,playlist_public
0,2tmzVHK0OfcbWWViZve0B6,Fall 2020,,Nicholas Drake,True
1,79tXL4BD7MnqvpMZtKDho9,Test Python Playlist 2020,This is a test playlist generated by Python,Nicholas Drake,True
2,5b4Ko7aAOpk8JT9qQnngOy,Pop Punk,,Amber Layn Miller,False
3,0RSKurPBF9Jd2W6zjAw6ZI,Chicago Unite At Night Complete Collection,Chicago nightly sing-along collection during C...,Chicago Unite At Night,False
4,7w3yeYpdXvclEmxFfsgVNr,Holiday-ish (feat. Dylan Minnette),,Nicholas Drake,True


In [44]:
playlist_id = playlist_df.loc[playlist_df['playlist_name'] == 'Test Python Playlist 2020', 'playlist_id'].tolist()[0]
playlist_id

'79tXL4BD7MnqvpMZtKDho9'

# Post song to playlist

In [ ]:
username = config.username
client_id = config.client_id
client_secret = config.client_secret
redirect_uri = 'http://localhost:7777/callback'
scope = 'playlist-modify-public, playlist-modify-private'

auth_token = util.prompt_for_user_token(username=username, 
                                   scope=scope, 
                                   client_id=client_id,   
                                   client_secret=client_secret,     
                                   redirect_uri=redirect_uri)

In [ ]:
playlist_id = '79tXL4BD7MnqvpMZtKDho9'
base_url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks"

# Must be in list to load into playlist
uris = spotify_urls_ls

# Authentication
# Parameters -- would be stored with authentication
headers = {
    "Content-Type":"application/json",
    "Authorization": f"Bearer {auth_token}"
}

request_body = json.dumps({
          "uris":uris
        })
# Make request
r = requests.post(url = base_url, data = request_body, headers=headers)
r.text